### 知识点掌握要求：
1. 通过gensim训练词向量 即Gensim工具的使用
 + 1.1 利用分词后的项目数据生成训练词向量用的训练数据
 + 1.2 保存词向量训练数据
 + 1.3 应用gensim中Word2Vec或Fasttext训练词向量
 + 1.4 保存训练好的词向量

2. 构建embedding_matrix

> 读取上步计算词向量和构建的`vocab`词表，以`vocab`中的`index`为`key`值构建`embedding_matrix`
这一部分主要是为了后边的模型，就是Embedding层的那一部分，即直接在构建Embedding层的时候导入训练好的词向量，这样模型就不用再花费较多的时间来处理

`eg: embedding_matrix[i] = [embedding_vector]`
3. 获得Embedding_matrix的两种方法
    - 定义函数获取
    - 直接调用

In [18]:
import re
import jieba
import pandas as pd
# 引入 word2vec
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText
from gensim.models import word2vec
import gensim
import numpy as np
import os
import pathlib

# 引入日志配置
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### 路径

In [19]:
# 获取根目录
root = pathlib.Path(os.path.abspath('__file__')).parent.parent
print(root)

# 数据路径
merger_data_path = os.path.join(root, 'data', 'merged_train_test_seg_data.csv')
# 词向量模型保存路径
save_model_path = os.path.join(root, 'data/wv', 'word2vec.model')
# 词向量矩阵保存路径
save_embedding_matrix_path=os.path.join(root, 'data', 'embedding_matrix.txt')

D:\Learning\Project\QA


### 训练模型

#### 使用word2vec训练

````
help(word2vec.Word2Vec)


里边有一个sg参数，通过设置该参数来指定是使用哪一个算法
sg : {0, 1}, optional
           Training algorithm: 1 for skip-gram; otherwise CBOW.
    
上一节讲到的一个softmax的优化的方法，这里使用下边这个参数就可以指定使用哪一个优化方法
hs : {0, 1}, optional
           If 1, hierarchical softmax will be used for model training. 分层softmax
           If 0, and `negative` is non-zero, negative sampling will be used.  负采样
```  

In [2]:
# 这里直接使用word2vec.Word2Vec这个包来训练word2vec这个模型,训练词向量
# 实例化word2vec模型为model_wv
model_wv = word2vec.Word2Vec(LineSentence(merger_data_path), sg=1,workers=8,min_count=5,size=200)
# LineSentence(merger_data_path): 一行一行的将该路径下的数据读进来
# sg=1: 使用Skip-Gram来构建word2vec
# workers=8: 使用8个进程来跑
# min_count=5：词频小于5的直接滤掉
# size=200：训练一个200维的词向量
# 这里word2vec如何定义是使用Skip-Gram还是CBOW，这里可以直接通过help(word2vec.Word2Vec)来查看
# 这里名字里边的wv就是word2vec的缩写

In [22]:
# 使用训练好的词向量模型计算相似度
model_wv.wv.most_similar(['奇瑞'], topn=10)

2020-05-17 11:31:23,928 : INFO : precomputing L2-norms of word weight vectors


[('瑞虎', 0.7469425797462463),
 ('瑞虎5', 0.7084462642669678),
 ('瑞虎3', 0.6410641074180603),
 ('东方之子', 0.6377476453781128),
 ('风云', 0.6346753239631653),
 ('江淮', 0.6338895559310913),
 ('名爵3', 0.632521390914917),
 ('风云2', 0.6258667707443237),
 ('瑞麒', 0.6176993250846863),
 ('4G16', 0.6092939376831055)]

#### 使用FastText训练

In [1]:
# 这里直接使用FastText这个包来训练FastText这个模型，训练词向量
# 实例化FastText模型为model_ft
model_ft = FastText(sentences=LineSentence(merger_data_path), workers=8, min_count=5, size=200)

In [24]:
# 使用训练好的词向量模型计算相似度
model_ft.wv.most_similar(['奇瑞'], topn=10)

2020-05-17 11:32:34,177 : INFO : precomputing L2-norms of word weight vectors
2020-05-17 11:32:34,215 : INFO : precomputing L2-norms of ngram weight vectors


[('奇瑞E5', 0.8964439034461975),
 ('奇瑞A1', 0.8838685750961304),
 ('奇瑞A5', 0.8812012672424316),
 ('奇瑞A3', 0.8743686676025391),
 ('瑞虎5', 0.8706405758857727),
 ('东南', 0.8699465990066528),
 ('奇瑞QQ6', 0.8604031801223755),
 ('瑞虎7', 0.8598191142082214),
 ('奇瑞QQ', 0.8594138622283936),
 ('东风皮卡', 0.8575624227523804)]

#### 模型保存

In [25]:
model_wv.save(save_model_path)

2020-05-17 11:32:36,263 : INFO : saving Word2Vec object under D:\Learning\Project\QA\data/wv\word2vec.model, separately None
2020-05-17 11:32:36,266 : INFO : not storing attribute vectors_norm
2020-05-17 11:32:36,267 : INFO : not storing attribute cum_table
2020-05-17 11:32:36,754 : INFO : saved D:\Learning\Project\QA\data/wv\word2vec.model


#### 模型的加载

In [26]:
model = word2vec.Word2Vec.load(save_model_path)

2020-05-17 11:32:36,759 : INFO : loading Word2Vec object from D:\Learning\Project\QA\data/wv\word2vec.model
2020-05-17 11:32:37,107 : INFO : loading wv recursively from D:\Learning\Project\QA\data/wv\word2vec.model.wv.* with mmap=None
2020-05-17 11:32:37,108 : INFO : setting ignored attribute vectors_norm to None
2020-05-17 11:32:37,108 : INFO : loading vocabulary recursively from D:\Learning\Project\QA\data/wv\word2vec.model.vocabulary.* with mmap=None
2020-05-17 11:32:37,109 : INFO : loading trainables recursively from D:\Learning\Project\QA\data/wv\word2vec.model.trainables.* with mmap=None
2020-05-17 11:32:37,110 : INFO : setting ignored attribute cum_table to None
2020-05-17 11:32:37,110 : INFO : loaded D:\Learning\Project\QA\data/wv\word2vec.model


#### 测试效果

In [27]:
# 使用保存好的模型计算相似度
model.wv.most_similar(['奇瑞'], topn=10)

2020-05-17 11:32:37,188 : INFO : precomputing L2-norms of word weight vectors


[('瑞虎', 0.7469425797462463),
 ('瑞虎5', 0.7084462642669678),
 ('瑞虎3', 0.6410641074180603),
 ('东方之子', 0.6377476453781128),
 ('风云', 0.6346753239631653),
 ('江淮', 0.6338895559310913),
 ('名爵3', 0.632521390914917),
 ('风云2', 0.6258667707443237),
 ('瑞麒', 0.6176993250846863),
 ('4G16', 0.6092939376831055)]

### 构建embedding_matrix
这里提前构建好词的embedding矩阵，这样的话后边进行模型训练的时候，就不用再进行词的Embedding了，直接将这里的Embedding矩阵导入即可。
就是一个根据id获取词向量的过程

#### 构建vocab
查看构建出来的vocab效果好不好：
在Gensim里边有这么一个方法：score，用来判断这个词向量好还是不好，也就是看一个它的输入和输出，它的输入是词，输出也是词。后边自己试一下

In [28]:
# 这个vocab是从model里边来，model里边是定义了一个词频参数，低于5的就滤掉，原先的语料就不用再去统计
# 这个框架就直接是低于5的就滤掉了，就直接实现了过滤掉了低频词
vocab = {word:index for index, word in enumerate(model_wv.wv.index2word)}
# 所以这里的表要定义成字典的形式，便于根据key得到value（词向量）
reverse_vocab = {index: word for index, word in enumerate(model_wv.wv.index2word)}

#### 获取embedding_matrix

#### 方法一
这种方法就完整的复现了第二次课里所描述的方法，就是直接拿到第 `i` 个词的词向量赋值给初始化的矩阵

In [29]:
def get_embedding_matrix(wv_model):
    # 获取vocab大小
    vocab_size = len(wv_model.wv.vocab)
    # 获取embedding维度:词向量的维度
    embedding_dim = wv_model.wv.vector_size
    print('vocab_size, embedding_dim:', vocab_size, embedding_dim)
    # 初始化词向量矩阵  shape和词向量矩阵一样  对应的行数就是 vocab_size(词的个数)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    # 这里的vocab_size就是词的个数
    # 按顺序填充
    for i in range(vocab_size):
        # 遍历整个初始化的词表，将对应 id 的词向量放入词表中
        embedding_matrix[i, :] = wv_model.wv[wv_model.wv.index2word[i]]
        # wv_model.wv.index2word[i]从第一个词开始依次输出词表里边的词，拿到它对应的向量，然后赋值给这个初始化全为0的numpy矩阵
        # 转换一下格式
        embedding_matrix = embedding_matrix.astype('float32')
    # 断言检查维度是否符合要求，是否是自己想要的大小
    assert embedding_matrix.shape == (vocab_size, embedding_dim)
    # 保存矩阵
    np.savetxt(save_embedding_matrix_path, embedding_matrix, fmt='%0.8f')
    print('embedding matrix extracted')
    return embedding_matrix

In [ ]:
embedding_matrix=get_embedding_matrix(model_wv)
print(embedding_matrix.shape)

vocab_size, embedding_dim: 32801 200


In [14]:
embedding_matrix.shape

(32905, 200)

#### 方法二
这里直接通过这里的方法，直接拿矩阵也可以

In [15]:
embedding_matrix_wv=model_wv.wv.vectors

In [16]:
embedding_matrix_wv.shape

(32905, 200)

#### 对比
对比两种方法得到的矩阵，所有的参数都是一样的，所以整体来说这个方法要好一些，直接拿取

In [17]:
embedding_matrix==embedding_matrix_wv

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [18]:
(embedding_matrix==embedding_matrix_wv).all()

True

##### Q1. 有没有一个标准的处理流程,怕前期数据处理影响后期项目效果? 
对于数据处理这个部分，一开始的方法可能会是一个比较low的方法，后边会不断的去完善数据处理这个部分，结合任务，不断的优化这个模块，这是一个不断修改，不断矫正的过程

### 参考

1. https://radimrehurek.com/gensim/models/word2vec.html 